# Sistemas de recomendação

Neste exemplo, iremos apresentar algumas formas de desenvolver um sistema de recomendação para filmes.

Será apresentada a metodologia para Filtragem Colaborativa.

Os dados e exemplos foram retirados de https://github.com/jeknov/movieRec.

Ao todo são 668 usuários e 10325 filmes.

## Carregar pacotes

In [1]:
library(tidyverse)
library(magrittr)
library(recommenderlab)
library(Matrix)
library(NMF)
library(NNLM)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.0.0     ✔ purrr   0.2.5
✔ tibble  1.4.2     ✔ dplyr   0.7.6
✔ tidyr   0.8.1     ✔ stringr 1.3.1
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘magrittr’

The following object is masked from ‘package:purrr’:

    set_names

The following object is masked from ‘package:tidyr’:

    extract

Loading required package: Matrix

Attaching package: ‘Matrix’

The following object is masked from ‘package:tidyr’:

    expand

Loading required package: arules

Attaching package: ‘arules’

The following object is masked from ‘package:dplyr’:

    recode

The following objects are masked from ‘package:base’:

    abbreviate, write

Loading required package: proxy

Attaching package: ‘proxy’

The following object is masked from ‘package:Matrix’

## Carregar dados

In [29]:
dados_ratings <- read_csv("/home/vm-data-science/dados/movie_ratings.csv")
dados_movies <- read_csv("/home/vm-data-science/dados/movies.csv")

Parsed with column specification:
cols(
  userId = col_integer(),
  movieId = col_integer(),
  rating = col_double(),
  timestamp = col_integer()
)
Parsed with column specification:
cols(
  movieId = col_integer(),
  title = col_character(),
  genres = col_character()
)


In [31]:
dados_ratings %<>% 
    sample_frac( 0.1 )

In [26]:
dados_ratings %>% head

userId,movieId,rating,timestamp
668,4018,2.0,1025855513
461,832,3.5,1193067126
283,474,3.0,868263866
603,1090,3.0,860499306
336,2918,4.5,1248029739
66,1625,4.0,961686150


In [32]:
dados_ratings %>% 
    distinct(userId) %>% dim

[1] 654   1

In [33]:
dados_ratings %>% dim

[1] 10534     4

In [11]:
dados_movies %>% head

movieId,title,genres
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
6,Heat (1995),Action|Crime|Thriller


In [17]:
dados_movies %>% dim

[1] 10329     3

## Análises

### Transformar em matriz de usuário/item

Os valores "NA" são os filmes que os usuários ainda não deram nota.

O objetivo é estimar estes valores pelos métodos que serão apresentados para sabermos se devemos recomendar ou não estes filmes.

In [34]:
user_item_matrix <- dados_ratings %>% 
    select( -timestamp ) %>% 
    spread( key = movieId, value = rating )

In [35]:
user_item_matrix[1:5, 1:5]

userId,1,2,3,4
1,NA,NA,NA,NA
2,NA,NA,NA,NA
3,NA,NA,NA,NA
4,NA,NA,NA,NA
5,NA,NA,NA,NA


## Algoritmos baseados em memória (*Memory Based Reasoning*)

Estes algoritmos, primeiramente, calculam a similaridade entre os usuários (*User based filtering*) ou itens (*Item based filtering*). Iremos apresentar ambos métodos.

Para realizar os cálculos, iremos utilizar as funções do pacote recommenderlab.

Neste pacote, primeiramente devemos transformar a matriz para o formato "realRatingMatrix".

In [36]:
user_item_matrix_reclab <- as.matrix(user_item_matrix)[,-1] %>% 
                                as(., "realRatingMatrix")

In [37]:
user_item_matrix_reclab

654 x 3830 rating matrix of class ‘realRatingMatrix’ with 10534 ratings.

### ***Item Based filtering***

Este método segue as etapas:

1 - Para cada 2 itens, calcule a similaridade entre eles.

2 - Para cada item, identifique os *k* itens mais similares. 

3 - Identifique os grupos de itens mais associados para cada usuário.

4 - Recomende o grupo de itens que estão mais associados ao usuário.

- **Matriz de distâncias**

A matriz de distância será calculada em relação aos filmes (3830 x 3830), iremos apresentar uma amostra.

A diagonal é zero porque a distância entre o item e ele mesmo é igual. O método de cálculo da distância foi o coseno.

In [38]:
similarity_items <- similarity(user_item_matrix_reclab[, 1:4], 
                               method = "cosine", 
                               which = "items")
as.matrix(similarity_items)

,1,2,3,4
1,0,1,1,1
2,1,0,NA,NA
3,1,NA,0,NA
4,1,NA,NA,0


- **Construção do modelo**

A matriz de similaridades é construída internamente no modelo.

Usamos k = 30, para buscar os 30 vizinhos mais próximos.

In [39]:
# cuidado - demora bastante
#item_based_rec_model <- Recommender( data = user_item_matrix_reclab, 
 #                                    method = "IBCF", # Item based
  #                                   parameter = list(k = 30))

In [40]:
#save( item_based_rec_model, file = "item_based_rec_model.RData")

In [41]:
# carregar o modelo salvo
load( "item_based_rec_model.RData" )

- **Uso do modelo**

In [42]:
numero_recomendações <- 5

In [43]:
item_based_recomendacoes <- predict( item_based_rec_model,
                                     user_item_matrix_reclab,
                                     n = numero_recomendações )

- **Recomendações de usuários para o filme**

In [82]:
dados_movies %>% 
    filter( movieId == 3 ) %>% 
    mutate( usersId = list(item_based_recomendacoes@items[[8]]) )

movieId,title,genres,usersId
3,Grumpier Old Men (1995),Comedy|Romance,"114, 191, 206, 484, 494"


### ***User based filtering***

- **Matriz de distâncias**

Como a matriz de distância entre os usuários será muito grande (654 x 654), iremos apresentar uma amostra.

A diagonal é zero porque a distância entre o usuário e ele mesmo é igual. O método de cálculo da distância foi o coseno.

In [56]:
similarity_users <- similarity(user_item_matrix_reclab[1:4, ], 
                               method = "cosine", 
                               which = "users")
as.matrix(similarity_users)

,1,2,3,4
1,0,NA,NA,NA
2,NA,0,NA,NA
3,NA,NA,0,NA
4,NA,NA,NA,0


- **Construção do modelo**

In [57]:
user_based_rec_model <- Recommender( data = user_item_matrix_reclab, 
                                     method = "UBCF" # User based 
                                   )

- **Uso do modelo**

In [58]:
numero_recomendacoes <- 10

In [59]:
user_based_recomendacoes <- predict( user_based_rec_model,
                                     user_item_matrix_reclab,
                                     n = numero_recomendações )

- **Recomendação de filmes para o usuário**

In [80]:
# filmes recomendados para o usuário 8
dados_movies %>% 
    filter( movieId %in% c(user_based_recomendacoes@items[[8]]) )

movieId,title,genres
40,"Cry, the Beloved Country (1995)",Drama
177,Lord of Illusions (1995),Horror
583,Dear Diary (Caro Diario) (1994),Comedy|Drama
2792,Airplane II: The Sequel (1982),Comedy


## Algoritmos baseados em modelos

Por meio da técnica de fatoração de matrizes, esses algoritmos preenchem os valores "NA" diretamente na matriz de usuários e itens.

Será utilizado o pacote NNLM combinado com o pacote NMF. Este pacote permite o uso de modelos baseados em *Alternating Least Squares*, estes proporcionam ganho de tempo e memória para estimar os *ratings*.

In [83]:
user_item_matrix_nnlm <- as.matrix(user_item_matrix)[,-1]

In [84]:
user_item_matrix_nnlm %>% head()

1,2,3,4,5,6,7,9,10,11,⋯,136020,138702,138863,139644,140247,140820,141305,144656,144976,146656
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [85]:
fatoracao_rec_model <- nnmf(user_item_matrix_nnlm, 
                            method = 'scd', 
                            loss = 'mse')

In [86]:
complete_user_item_matrix <- fatoracao_rec_model$W %*% fatoracao_rec_model$H

In [87]:
complete_user_item_matrix %>% head()

1,2,3,4,5,6,7,9,10,11,⋯,136020,138702,138863,139644,140247,140820,141305,144656,144976,146656
3.879486,3.769182,3.490703,2.439220,3.569014,3.849695,3.852912,3.006848,3.665552,3.545650,⋯,3.693775,4.249371,2.443081,4.286129,4.112343,4.158994,3.500481,3.768083,3.830962,4.112343
3.822143,3.713470,3.439107,2.403166,3.516260,3.792792,3.795962,2.962403,3.611372,3.493242,⋯,3.639177,4.186560,2.406969,4.222776,4.051558,4.097520,3.448740,3.712387,3.774337,4.051558
4.035885,3.921134,3.631428,2.537556,3.712896,4.004892,4.008240,3.128067,3.813327,3.688591,⋯,3.842687,4.420681,2.541572,4.458922,4.278129,4.326661,3.641600,3.919991,3.985405,4.278129
3.910564,3.799377,3.518667,2.458761,3.597604,3.880534,3.883777,3.030935,3.694917,3.574054,⋯,3.723365,4.283412,2.462652,4.320465,4.145286,4.192311,3.528523,3.798269,3.861652,4.145286
2.111282,2.051253,1.899700,1.327465,1.942318,2.095069,2.096820,1.636378,1.994856,1.929603,⋯,2.010215,2.312579,1.329566,2.332584,2.238006,2.263395,1.905021,2.050655,2.084875,2.238006
3.744082,3.637627,3.368868,2.354085,3.444445,3.715330,3.718435,2.901901,3.537615,3.421897,⋯,3.564852,4.101056,2.357810,4.136532,3.968811,4.013834,3.378304,3.636567,3.697251,3.968811


Podemos combinar as duas matrizes para obter os *ratings* dos filmes ainda não foram assistidos e poderão ser recomendados.

In [88]:
matriz_recomendacoes <- ( is.na(user_item_matrix_nnlm) == TRUE ) * round(complete_user_item_matrix, 2)

In [89]:
matriz_recomendacoes %>% head()

1,2,3,4,5,6,7,9,10,11,⋯,136020,138702,138863,139644,140247,140820,141305,144656,144976,146656
3.88,3.77,3.49,2.44,3.57,3.85,3.85,3.01,3.67,3.55,⋯,3.69,4.25,2.44,4.29,4.11,4.16,3.50,3.77,3.83,4.11
3.82,3.71,3.44,2.40,3.52,3.79,3.80,2.96,3.61,3.49,⋯,3.64,4.19,2.41,4.22,4.05,4.10,3.45,3.71,3.77,4.05
4.04,3.92,3.63,2.54,3.71,4.00,4.01,3.13,3.81,3.69,⋯,3.84,4.42,2.54,4.46,4.28,4.33,3.64,3.92,3.99,4.28
3.91,3.80,3.52,2.46,3.60,3.88,3.88,3.03,3.69,3.57,⋯,3.72,4.28,2.46,4.32,4.15,4.19,3.53,3.80,3.86,4.15
2.11,2.05,1.90,1.33,1.94,2.10,2.10,1.64,1.99,1.93,⋯,2.01,2.31,1.33,2.33,2.24,2.26,1.91,2.05,2.08,2.24
3.74,3.64,3.37,2.35,3.44,3.72,3.72,2.90,3.54,3.42,⋯,3.56,4.10,2.36,4.14,3.97,4.01,3.38,3.64,3.70,3.97


Associamos novamente com os usuários.

In [91]:
matriz_recomendacoes <- cbind( user_item_matrix$userId, data.frame(matriz_recomendacoes) )

In [93]:
# alguns ajustes
matriz_recomendacoes %<>% 
    rename( userId = `user_item_matrix$userId` )

Podemos ajustar para organizar um banco de dados ordenado com as possíveis recomendações.

In [94]:
banco_ratings <- matriz_recomendacoes %>% 
    gather( key = movieId, value = ratings, -userId  ) %>% 
    mutate( movieId = as.integer(str_extract(movieId, "[0-9]") ) ) %>% 
    filter( ratings > 0 )

Combinamos com o banco de filmes.

In [95]:
banco_ratings %<>% 
    left_join(., y = dados_movies,
              by = "movieId" )

In [96]:
banco_ratings %>% head()

userId,movieId,ratings,title,genres
1,1,3.88,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,1,3.82,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,1,4.04,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,1,3.91,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
5,1,2.11,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
6,1,3.74,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


Podemos ver as 5 melhores recomendações para o usuário 8.

In [98]:
banco_ratings %>% 
    filter( userId == 8 ) %>% 
    arrange( desc(ratings) ) %>% 
    head(5)

userId,movieId,ratings,title,genres
8,3,10.55,Grumpier Old Men (1995),Comedy|Romance
8,4,6.69,Waiting to Exhale (1995),Comedy|Drama|Romance
8,6,6.58,Heat (1995),Action|Crime|Thriller
8,1,6.58,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
8,5,6.54,Father of the Bride Part II (1995),Comedy
